In [1]:
import time
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import torch, pykitti
import numpy as np
import sys
# sys.path.append("..")

from data_loader.kitti_odometry_dataset import KittiOdometryDataset
from model.monorec.monorec_model import MonoRecModel
from utils import unsqueezer, map_fn, to

target_image_size = (256, 512)

device = "cuda:0" if torch.cuda.is_available() else "cpu"

dataset = KittiOdometryDataset("example/data/kitti", sequences=["07"], target_image_size=target_image_size, frame_count=2,
                               depth_folder="image_depth_annotated", lidar_depth=True, use_dso_poses=True,
                               use_index_mask=None)
dataset._dataset_sizes = [1000]
dataset._datasets[0].cam2_files = [f"example/data/kitti/sequences/07/image_2/{i:06d}.png" for i in range(dataset._dataset_sizes[0])]
dataset._datasets[0].cam3_files = [f"example/data/kitti/sequences/07/image_3/{i:06d}.png" for i in range(dataset._dataset_sizes[0])]
# print(dataset._datasets[0].cam3_files) # cam? corresponds to image_? folder

batch, depth = dataset.__getitem__(164)


self._offset=5



In [2]:
# len(batch['frames'])
ind=169
Img2=dataset._datasets[0].get_cam2(ind)
# Img2.show()




c:\ProgramData\Anaconda3\envs\torch_17\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
